In [1]:
import pandas as pd
import os
import re
import time
from haversine import haversine
import pymysql

from tqdm import tqdm

In [2]:
# 위치데이터
pymydb = pymysql.connect(
    user='kk-bigdata', 
    passwd='Kgu0318885100!', 
    host='172.16.131.2',  
    charset='utf8'
)

cursor = pymydb.cursor()

sql = "select delivery.SN, delivery.PROCESS_DT, location.DLVR_STATPNT_LA_YCRD, location.DLVR_STATPNT_LO_XCRD, location.DLVR_DSTN_LA_YCRD, location.DLVR_DSTN_LO_XCRD from kt_upload_20210501.KGU_DLVR_CALL_INFO as delivery left join delivery_analysis_20201201.ANALYSIS_CALL_INFO as location on delivery.SN = location.SN where delivery.PROCESS_DT > '2019-11-30' and delivery.PROCESS_DT < '2020-06-01';"

cursor.execute(sql)

df = cursor.fetchall()
pymydb.close()

# 배달데이터 확인
df = pd.DataFrame(df)

NameError: name 'location_df' is not defined

In [3]:
df.columns = ['SN', 'PROCESS_DT', 'DLVR_STATPNT_LA_YCRD', 'DLVR_STATPNT_LO_XCRD', 'DLVR_DSTN_LA_YCRD', 'DLVR_DSTN_LO_XCRD']

In [4]:
distance_list = []

for x in tqdm(range(len(df))) :
    start_point = (df['DLVR_STATPNT_LA_YCRD'].iloc[x], df['DLVR_STATPNT_LO_XCRD'].iloc[x])
    end_point = (df['DLVR_DSTN_LA_YCRD'].iloc[x], df['DLVR_DSTN_LO_XCRD'].iloc[x])
    distance_list.append(haversine(start_point, end_point, unit = 'km'))
distance_list[:5]

100%|█████████████████████████████████████████████████████████████████████| 5843946/5843946 [04:20<00:00, 22398.37it/s]


[1.295235685114082,
 1.1131301627833337,
 1.1656753891900915,
 0.6657986472964232,
 2.033422918603361]

In [5]:
new_df = df[['SN']]
new_df['DISTANCE'] = distance_list

C:\Users\User\Anaconda3\envs\psj\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [6]:
display(new_df.head())

,SN,DISTANCE
0,3486927,1.295236
1,3486929,1.113130
2,3486930,1.165675
3,3486931,0.665799
4,3486932,2.033423


In [7]:
# 데이터 저장
new_df.to_csv("./data/거리데이터2.csv", index=False)